# **Machine Learning**

### Importación de Librerías

In [107]:
import pandas as pd
import numpy as np
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#!pip install fuzzywuzzy
#!pip install python-Levenshtein
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## **Ingesta de Datos limpios**

In [108]:
# leer los datos que fueron previamente analizados en el EDA
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/PI_MLOps/movies_clean.parquet')

In [109]:
# Mostrar información del número de columnas, filas, tipos de datos y valores faltantes presentes en "df"
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45431 entries, 0 to 45430
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45431 non-null  int32         
 1   genres                44915 non-null  object        
 2   id                    45431 non-null  int32         
 3   original_language     45431 non-null  object        
 4   overview              45431 non-null  object        
 5   popularity            45428 non-null  float64       
 6   production_companies  33562 non-null  object        
 7   production_countries  39151 non-null  object        
 8   release_date          45431 non-null  datetime64[ns]
 9   revenue               45431 non-null  float64       
 10  runtime               45412 non-null  float64       
 11  status                45347 non-null  object        
 12  tagline               20397 non-null  object        
 13  title           

In [110]:
# Seleccionamos solo las filas duplicadas en la columna 'title'
duplicados = df[df.duplicated(['title'], keep=False)]

# Ordenamos el DataFrame por el valor de la columna 'title'
duplicados = duplicados.sort_values('title')

duplicados[['title', 'vote_count', 'release_year', 'production_companies']]

,title,vote_count,release_year,production_companies
42658,10 Minutes,2.0,2014,None
24402,10 Minutes,7.0,2002,None
15194,12 Angry Men,59.0,1997,MGM Television
1161,12 Angry Men,2130.0,1957,"United Artists,Orion-Nova Productions"
32753,12 Chairs,16.0,1976,Ekran
...,...,...,...,...
11643,Zodiac,2080.0,2007,"Paramount Pictures,Warner Bros.,Phoenix Pictures"
37870,Zoom,25.0,2015,"Rhombus Media,O2 Filmes"
11168,Zoom,140.0,2006,"Columbia Pictures,Revolution Studios"
5760,Zulu,137.0,1964,Diamond Films


**Observación:**

Hay títulos duplicados de películas que fueron lanzadas por diferentes compañías productoras en años distintos y que de hecho presentan votaciones distintas. Esto es un dato importante a la hora de construir el modelo de Sistema de Recomendación de Películas.

In [111]:
df.columns

Index(['budget', 'genres', 'id', 'original_language', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'languages', 'director', 'actors', 'overview_clean',
       'release_year', 'return'],
      dtype='object')

## **Consideraciones Iniciales**

A continuación se presentarán diferentes modelos de Sistema de Recomendación de Películas basados en columnas analizadas en el EDA y siguiendo enfoques comúnmente utilizados. Debido a las limitaciones computacionales de Google Colab, se adaptarán algunos parámetros y métodos, y el código se presentará estratégicamente separado en diferentes celdas en lugar de en una única función.

Es importante tener en cuenta que la selección final del modelo que se utilizará en este proyecto estará sujeta a las limitaciones del plan de desarrollador gratuito de Render que ofrece 512 MB de memoria RAM. Este servicio se utilizará para implementar el modelo.

## **Modelo de Recomendación de Película basado en contenido**

Un modelo de sistema de recomendación de películas basado en contenido es un algoritmo que analiza las características de las películas (como el género, el reparto, la sinopsis, etc.) para recomendar películas similares.

Este tipo de modelo utiliza técnicas de procesamiento de lenguaje natural (NLP) y aprendizaje automático para analizar los datos de las películas y crear un perfil de cada película, que se utiliza para calcular la similitud entre ellas.

### Técnica a utilizar

Para los siguientes modelos se usará:

- **TF-IDF**: Esta técnica se utiliza para asignar un peso a cada palabra en un texto en función de su importancia relativa en dicha descripción y en el corpus completo. El TF-IDF considera tanto la frecuencia de aparición de la palabra en el texto (TF) como la rareza de la palabra en el corpus (IDF). El resultado es un vector de características numéricas que representa el texto.

- **Similitud del coseno**: La similitud del coseno es una medida utilizada para comparar la similitud entre dos vectores de características. Para este caso, serán las películas más similares.

### Feature Engineering

In [123]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [124]:
# Tokenizar el texto en la columna 'overview'
df['tokens'] = df['overview_clean'].apply(word_tokenize)
# Crear una lista de stopwords en inglés
stop_words = set(stopwords.words('english'))

In [125]:
# Filtrar las stopwords y convertir el texto en minúsculas
df['keywords'] = df['tokens'].apply(lambda tokens: [token.lower() for token in tokens if token.lower() not in stop_words])

In [126]:
# Convertir la lista de palabras clave en una cadena separada por comas
df['keywords'] = df['keywords'].apply(lambda keywords: ', '.join(keywords))

In [127]:
df['keywords'].head()

0    led, woody, andys, toy, live, happily, room, a...
1    sibling, judy, peter, discover, enchanted, boa...
2    family, wedding, reignites, ancient, feud, nex...
3    cheated, mistreated, stepped, woman, holding, ...
4    george, bank, ha, recovered, daughter, wedding...
Name: keywords, dtype: object

### **Modelo 1**

Considerando únicamente la descripciones de las películas.

Campos a utiliar: ```title```, ```overview```

In [291]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo1 = df[['title', 'overview_clean']].copy()

In [292]:
# Se crea una instancia de la clase TfidfVectorizer con los parámetros deseados
tfidf_1 = TfidfVectorizer(stop_words="english", ngram_range = (1, 2))
# Aplicar la transformación TF-IDF al texto contenido en la columna "overview" del dataframe 'modelo1'
tfidf_matriz_1 = tfidf_1.fit_transform(modelo1['overview_clean'])

In [245]:
# Cantidad de registros y columnas en tfidf_matriz_1
tfidf_matriz_1.shape

(45431, 1012562)

In [293]:
def get_recomendacion_m1(titulo):
  # Se crea un objeto 'indices' que mapea los títulos de las películas a sus índices correspondientes en el DataFrame 'modelo1'
  indices = pd.Series(modelo1.index, index=modelo1['title']).drop_duplicates()
  # Se busca el índice de la película de entrada utilizando el objeto 'indices'
  idx = pd.Series(indices[titulo]) if titulo in indices else None
  # Si la película de entrada no se encuentra en el DataFrame, se devuelve un mensaje de error
  if idx is None:
    return "Película no encontrada"
  # Si el título de la película está duplicado, se devuelve el índice de la primera aparición del título en el DataFrame
  if modelo1.duplicated(['title']).any():
    primer_idx = modelo1[modelo1['title'] == titulo].index[0]
    if not idx.equals(pd.Series(primer_idx)):
      idx = pd.Series(primer_idx)
  # Se calcula la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
  simil = sorted(enumerate(cosine_similarity(tfidf_matriz_1[idx], tfidf_matriz_1).flatten()), key=lambda x: x[1], reverse=True)[1:6]
  # Se obtienen los títulos de las películas más similares utilizando el índice de cada película
  recomendaciones = modelo1.iloc[[i[0] for i in simil], :]['title'].tolist()
  # Se devuelve la lista de títulos de las películas recomendadas
  return recomendaciones

In [121]:
# Prueba #1
get_recomendacion_m1('The Dark Knight Rises')

['The Dark Knight',
 'Batman Forever',
 'Batman',
 'Batman Returns',
 'Batman: Under the Red Hood']

In [10]:
# Prueba #2
get_recomendacion_m1('The Avengers')

['The Work and the Glory',
 "Sir Arne's Treasure",
 'Monty Python and the Holy Grail',
 'A Beast at Bay',
 'Enigma']

In [11]:
# Prueba #3 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m1('Wonder Woman')

['Wide Eyed and Legless',
 'Trevor Noah: The Daywalker',
 'The Machinist',
 'Diana',
 'Our Dancing Daughters']

In [12]:
# Prueba #4 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m1('Titanic')

['Grantham and Rose',
 'Take Care of the Women!',
 'The Legend of 1900',
 'Raise the Titanic',
 'Beyond the Poseidon Adventure']

In [247]:
# Prueba #5
get_recomendacion_m1('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Star Wars: The Force Awakens',
 'Return of the Jedi',
 'Samson and the Seven Miracles of the World']

In [294]:
# Prueba #6
get_recomendacion_m1('The Hunger Games')

['The Hunger Games: Catching Fire',
 'The Hunger Games: Mockingjay - Part 2',
 'The Starving Games',
 'The Hungover Games',
 'Indie Game: The Movie']

In [295]:
modelo1 = get_recomendacion_m1('The Hunger Games')

**Conclusión:**

El hecho de que el modelo pueda identificar correctamente que "The Dark Knight Rises" es una película de Batman y recomendar otras películas similares de Batman es un buen indicio de que el modelo está funcionando adecuadamente para ciertos casos. Sin embargo, el que no pueda hacer recomendaciones precisas para películas como "The Avengers" y "Titanic" sugiere que el modelo puede tener limitaciones importantes en su capacidad para hacer recomendaciones específicas y personalizadas.

Una de las principales limitaciones del modelo es que no tiene en cuenta otras características importantes como los actores, los directores y el género de las películas. Estos son factores importantes que pueden influir en las preferencias de los usuarios y, por lo tanto, es probable que limiten la capacidad del modelo para hacer mejores recomendaciones.


### **Modelo 2**

Para este caso sustituiremos las descripciones de las películas por las keywords, para detectar si hay alguna diferencia aparente

Campos a utilizar: ```title```, ```keywords```

In [286]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo2 = df[['title', 'keywords']].copy()
# Se realiza un preprocesamiento en la columna "keywords" para separar los géneros y convertirlos en palabras individuales
modelo2['keywords'] = modelo2['keywords'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))

In [287]:
# Se crea una instancia de la clase TfidfVectorizer con los parámetros deseados
tfidf_2 = TfidfVectorizer(stop_words="english", ngram_range = (1, 2))
# Aplicar la transformación TF-IDF al texto contenido en la columna "overview" del dataframe 'modelo2'
tfidf_matriz_2 = tfidf_2.fit_transform(modelo2['keywords'])

In [177]:
# Cantidad de registros y columnas en tfidf_matriz_2
tfidf_matriz_2.shape

(45431, 1010451)

In [288]:
def get_recomendacion_m2(titulo):
  # Se crea un objeto 'indices' que mapea los títulos de las películas a sus índices correspondientes en el DataFrame 'modelo2'
  indices = pd.Series(modelo2.index, index=modelo2['title']).drop_duplicates()
  # Se busca el índice de la película de entrada utilizando el objeto 'indices'
  idx = pd.Series(indices[titulo]) if titulo in indices else None
  # Si la película de entrada no se encuentra en el DataFrame, se devuelve un mensaje de error
  if idx is None:
    return "Película no encontrada"
  # Si el título de la película está duplicado, se devuelve el índice de la primera aparición del título en el DataFrame
  if modelo2.duplicated(['title']).any():
    primer_idx = modelo2[modelo2['title'] == titulo].index[0]
    if not idx.equals(pd.Series(primer_idx)):
      idx = pd.Series(primer_idx)
  # Se calcula la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
  simil = sorted(enumerate(cosine_similarity(tfidf_matriz_2[idx], tfidf_matriz_2).flatten()), key=lambda x: x[1], reverse=True)[1:6]
  # Se obtienen los títulos de las películas más similares utilizando el índice de cada película
  recomendaciones = modelo2.iloc[[i[0] for i in simil], :]['title'].tolist()
  # Se devuelve la lista de títulos de las películas recomendadas
  return recomendaciones

In [132]:
# Prueba #1
get_recomendacion_m2('The Dark Knight Rises')

['The Dark Knight',
 'Batman Forever',
 'Batman',
 'Batman Returns',
 'Batman: Under the Red Hood']

In [25]:
# Prueba #2
get_recomendacion_m2('The Avengers')

['The Work and the Glory',
 'Monty Python and the Holy Grail',
 "Sir Arne's Treasure",
 'A Beast at Bay',
 'Enigma']

In [26]:
# Prueba #3 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m2('Wonder Woman')

['Wide Eyed and Legless',
 'Trevor Noah: The Daywalker',
 'The Machinist',
 'Diana',
 'Our Dancing Daughters']

In [27]:
# Prueba #4 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m2('Titanic')

['Grantham and Rose',
 'Take Care of the Women!',
 'The Legend of 1900',
 'Raise the Titanic',
 'Beyond the Poseidon Adventure']

In [252]:
# Prueba #5
get_recomendacion_m2('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Star Wars: The Force Awakens',
 'Return of the Jedi',
 'Samson and the Seven Miracles of the World']

In [289]:
# Prueba #6
get_recomendacion_m2('The Hunger Games')

['The Hunger Games: Catching Fire',
 'The Hunger Games: Mockingjay - Part 2',
 'The Starving Games',
 'The Hungover Games',
 'Indie Game: The Movie']

In [290]:
modelo2 = get_recomendacion_m2('The Hunger Games')

**Observación:**

A parentemente no hay ninguna diferencia entre usar el atributo ```overview_clean``` y ```keywords```

### **Modelo 3**

Para este caso agregaremos información respecto a los géneros de las películas

Campos a utilizar: ```title```, ```overview```, ```genres```

In [281]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo3 = df[['title', 'overview_clean', 'genres']].copy()
# Se realiza un preprocesamiento en la columna "genres" para separar los géneros y convertirlos en palabras individuales
modelo3['genres'] = modelo3['genres'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))

In [282]:
# Se crea una instancia de la clase TfidfVectorizer con los parámetros deseados
tfidf_3 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
# Incluir la columna 'vote_count' en la concatenación de columnas
modelo3['concatenado'] = modelo3['overview_clean'] + ' ' + modelo3['genres']
# Aplicar la transformación TF-IDF al texto contenido en las columnas "overview_clean" y "genres" del dataframe 'modelo3'
tfidf_matriz_3 = tfidf_3.fit_transform(modelo3['concatenado'])

In [181]:
# Cantidad de registros y columnas en tfidf_matriz_3
tfidf_matriz_3.shape

(45431, 1033282)

In [283]:
def get_recomendacion_m3(titulo):
  # Se crea un objeto 'indices' que mapea los títulos de las películas a sus índices correspondientes en el DataFrame 'modelo3'
  indices = pd.Series(modelo3.index, index=modelo3['title']).drop_duplicates()
  # Se busca el índice de la película de entrada utilizando el objeto 'indices'
  idx = pd.Series(indices[titulo]) if titulo in indices else None
  # Si la película de entrada no se encuentra en el DataFrame, se devuelve un mensaje
  if idx is None:
    return "Película no encontrada"
  # Si el título de la película está duplicado, se devuelve el índice de la primera aparición del título en el DataFrame
  if modelo3.duplicated(['title']).any():
    primer_idx = modelo3[modelo3['title'] == titulo].index[0]
    if not idx.equals(pd.Series(primer_idx)):
      idx = pd.Series(primer_idx)
  # Se calcula la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
  simil = sorted(enumerate(cosine_similarity(tfidf_matriz_3[idx], tfidf_matriz_3).flatten()), key=lambda x: x[1], reverse=True)[1:6]
  # Se obtienen los títulos de las películas más similares utilizando el índice de cada película
  recomendaciones = modelo3.iloc[[i[0] for i in simil], :]['title'].tolist()
  # Se devuelve la lista de títulos de las películas recomendadas
  return recomendaciones

In [137]:
# Prueba #1
get_recomendacion_m3('The Dark Knight Rises')

['The Dark Knight',
 'Batman Forever',
 'Batman Returns',
 'Batman',
 'Batman: Under the Red Hood']

In [43]:
# Prueba #2
get_recomendacion_m3('The Avengers')

['The Work and the Glory',
 'Monty Python and the Holy Grail',
 "Sir Arne's Treasure",
 'A Beast at Bay',
 'Enigma']

In [44]:
# Prueba #3 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m3('Wonder Woman')

['Wide Eyed and Legless',
 'Trevor Noah: The Daywalker',
 'The Machinist',
 'Diana',
 'Our Dancing Daughters']

In [45]:
# Prueba #4 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m3('Titanic')

['Grantham and Rose',
 'Titanic',
 'Take Care of the Women!',
 'Raise the Titanic',
 'The Legend of 1900']

In [46]:
# Prueba #5
get_recomendacion_m3('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Return of the Jedi',
 'Star Wars: The Force Awakens',
 'Samson and the Seven Miracles of the World']

In [284]:
# Prueba #6
get_recomendacion_m3('The Hunger Games')

['The Hunger Games: Catching Fire',
 'The Hunger Games: Mockingjay - Part 2',
 'The Starving Games',
 'The Hungover Games',
 'The Hunger Games: Mockingjay - Part 1']

In [285]:
modelo3 = get_recomendacion_m3('The Hunger Games')

**Conclusión:**

En comparación con el modelo anterior, el modelo actual muestra una mejora en el orden de las recomendaciones. Sin embargo, aún existen oportunidades para mejorar la precisión y personalización de las recomendaciones.

Para abordar esta limitación, se explorará la posibilidad de agregar más características resaltantes al modelo, como los actores, directores y géneros de las películas. Estas características pueden ser importantes para los usuarios al elegir películas y pueden ayudar al modelo a hacer recomendaciones más precisas y personalizadas.

### **Modelo 4**

Se verá ahora si el añadir el atributo ```director``` mejoren las recomendaciones

Campos a utilizar: ```title```, ```overview```, ```genres```, ```director```

In [276]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo4 = df[['title', 'overview_clean', 'genres', 'director']].copy()
# Se realiza un preprocesamiento en la columna "genres" para separar los géneros y convertirlos en palabras individuales
modelo4['genres'] = modelo4['genres'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
# Se realiza un preprocesamiento en la columna "director" para separar los géneros y convertirlos en palabras individuales
modelo4['director'] = modelo4['director'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))

In [277]:
# Se crea una instancia de la clase TfidfVectorizer con los parámetros deseados
tfidf_4 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
# Aplicar la transformación TF-IDF al texto contenido en las columnas "overview_clean", "genres" y "director" del dataframe 'modelo4'
tfidf_matriz_4 = tfidf_4.fit_transform(modelo4['overview_clean'] + ' ' + modelo4['genres'] + ' ' + modelo4['director'])

In [260]:
# Cantidad de registros y columnas en tfidf_matriz_4
tfidf_matriz_4.shape

(45431, 1081581)

In [278]:
def get_recomendacion_m4(titulo):
  # Se crea un objeto 'indices' que mapea los títulos de las películas a sus índices correspondientes en el DataFrame 'modelo4'
  indices = pd.Series(modelo4.index, index=modelo4['title']).drop_duplicates()
  # Se busca el índice de la película de entrada utilizando el objeto 'indices'
  idx = pd.Series(indices[titulo]) if titulo in indices else None
  # Si la película de entrada no se encuentra en el DataFrame, se devuelve un mensaje de error
  if idx is None:
    return "Película no encontrada"
  # Si el título de la película está duplicado, se devuelve el índice de la primera aparición del título en el DataFrame
  if modelo4.duplicated(['title']).any():
    primer_idx = modelo4[modelo4['title'] == titulo].index[0]
    if not idx.equals(pd.Series(primer_idx)):
      idx = pd.Series(primer_idx)
  # Se calcula la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
  simil = sorted(enumerate(cosine_similarity(tfidf_matriz_4[idx], tfidf_matriz_4).flatten()), key=lambda x: x[1], reverse=True)[1:6]
  # Se obtienen los títulos de las películas más similares utilizando el índice de cada película
  recomendaciones = modelo4.iloc[[i[0] for i in simil], :]['title'].tolist()
  # Se devuelve la lista de títulos de las películas recomendadas
  return recomendaciones

In [142]:
# Prueba #1
get_recomendacion_m4('The Dark Knight Rises')

['The Dark Knight',
 'Batman Forever',
 'Batman Returns',
 'Batman',
 'Batman Begins']

In [100]:
# Prueba #2
get_recomendacion_m4('The Avengers')

['The Work and the Glory',
 'The Right Kind of Wrong',
 'Benny & Joon',
 'Diabolique',
 "Sir Arne's Treasure"]

In [101]:
# Prueba #3 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m4('Wonder Woman')

['DC Showcase: Catwoman',
 'Green Lantern: First Flight',
 'Wide Eyed and Legless',
 'Green Lantern: Emerald Knights',
 'Superman: Doomsday']

In [102]:
# Prueba #4 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m4('Titanic')

['Ghosts of the Abyss',
 'Grantham and Rose',
 'Titanic',
 'The Legend of 1900',
 'Deadly Voyage']

In [262]:
# Prueba #5
get_recomendacion_m4('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Return of the Jedi',
 'Star Wars: The Force Awakens',
 'Star Wars: Episode I - The Phantom Menace']

In [279]:
# Prueba #6
get_recomendacion_m4('The Hunger Games')

['The Hunger Games: Catching Fire',
 'The Hunger Games: Mockingjay - Part 2',
 'The Starving Games',
 'The Hungover Games',
 'The Hunger Games: Mockingjay - Part 1']

In [280]:
modelo4 = get_recomendacion_m4('The Hunger Games')

**Conclusión:**

Al agregar la variable director, los resultados comienzan a presentar mejorias especialmente en las recomendaciones para Titanic y Wonder Woman. Veamos que sucede si se agregar una variable más

### **Modelo 5**

Se finalizará con la información de los actores ```actors```

Campos a utilizar: ```title```, ```overview```, ```genres```, ```director```, ```actors```

In [271]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo5 = df[['title', 'actors', 'overview_clean', 'genres', 'director']].copy()
# Se realiza un preprocesamiento en la columna "genres" para separar los géneros y convertirlos en palabras individuales
modelo5['genres'] = modelo5['genres'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
# Se realiza un preprocesamiento en la columna "director" para separar los géneros y convertirlos en palabras individuales
modelo5['director'] = modelo5['director'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
# Se realiza un preprocesamiento en la columna "actors" para separar los géneros y convertirlos en palabras individuales
modelo5['actors'] = modelo5['actors'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
# Se realiza un preprocesamiento en la columna "keywords" para separar los géneros y convertirlos en palabras individuales
# modelo5['keywords'] = modelo5['keywords'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))

In [272]:
# Se crea una instancia de la clase TfidfVectorizer con los parámetros deseados
tfidf_5 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
# Aplicar la transformación TF-IDF al texto contenido en las columnas "overview_clean", "genres", "actors" y "director" del dataframe 'modelo5'
tfidf_matriz_5 = tfidf_5.fit_transform(modelo5['overview_clean'] + ' ' + modelo5['genres'] + ' ' + modelo5['director'] + ' ' + modelo5['actors'])

In [189]:
# Cantidad de registros y columnas en tfidf_matriz_5
tfidf_matriz_5.shape

(45431, 1863025)

In [273]:
def get_recomendacion_m5(titulo):
  # Se crea un objeto 'indices' que mapea los títulos de las películas a sus índices correspondientes en el DataFrame 'modelo5'
  indices = pd.Series(modelo5.index, index=modelo5['title']).drop_duplicates()
  # Se busca el índice de la película de entrada utilizando el objeto 'indices'
  idx = pd.Series(indices[titulo]) if titulo in indices else None
  # Si la película de entrada no se encuentra en el DataFrame, se devuelve un mensaje de error
  if idx is None:
    return "Película no encontrada"
  # Si el título de la película está duplicado, se devuelve el índice de la primera aparición del título en el DataFrame
  if modelo5.duplicated(['title']).any():
    primer_idx = modelo5[modelo5['title'] == titulo].index[0]
    if not idx.equals(pd.Series(primer_idx)):
      idx = pd.Series(primer_idx)
  # Se calcula la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
  simil = sorted(enumerate(cosine_similarity(tfidf_matriz_5[idx], tfidf_matriz_5).flatten()), key=lambda x: x[1], reverse=True)[1:6]
  # Se obtienen los títulos de las películas más similares utilizando el índice de cada película
  recomendaciones = modelo5.iloc[[i[0] for i in simil], :]['title'].tolist()
  # Se devuelve la lista de títulos de las películas recomendadas
  return recomendaciones

In [147]:
# Prueba #1
get_recomendacion_m5('The Dark Knight Rises')

['The Dark Knight',
 'Batman Begins',
 'Interstellar',
 'The Wolf of Wall Street',
 'Inception']

In [84]:
# Prueba #2
get_recomendacion_m5('The Avengers')

['The Work and the Glory',
 'Hysterical Blindness',
 'Eddie Izzard: Unrepeatable',
 'Eddie Izzard: Circle',
 'The Frightened City']

In [85]:
# Prueba #3 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m5('Wonder Woman')

['DC Showcase: Catwoman',
 'Superman/Batman: Apocalypse',
 'Green Lantern: Emerald Knights',
 'Green Lantern: First Flight',
 'Superman: Doomsday']

In [86]:
# Prueba #4 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m5('Titanic')

['Aliens of the Deep',
 'Titanic: The Final Word with James Cameron',
 'Ghosts of the Abyss',
 'The Bounty',
 'The Departed']

In [267]:
# Prueba #5
get_recomendacion_m5('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Return of the Jedi',
 'Empire of Dreams: The Story of the Star Wars Trilogy',
 'Elstree 1976']

In [274]:
# Prueba #6
get_recomendacion_m5('The Hunger Games')

['The Hunger Games: Catching Fire',
 'The Hunger Games: Mockingjay - Part 2',
 'The Hunger Games: Mockingjay - Part 1',
 "Buster's Mal Heart",
 'Quarantine 2: Terminal']

**Conclusión:**

La inclusión del atributo "actores" ha tenido un impacto significativo en la calidad de las recomendaciones de películas. Ahora se están generando recomendaciones más personalizadas que toman en cuenta la aparición de los actores y directores en las películas, además de la descripción y el género. Por ejemplo, en el caso de The Dark Knight Rises, el modelo ya no sólo recomienda películas relacionadas con Batman, sino que también sugiere otros tipos de películas con similitudes.

Aunque esto puede considerarse una mejora con respecto a los modelos anteriores, aún hay limitaciones en la capacidad del modelo debido a la falta de información sobre el usuario y su historial de películas consumidas. Esto significa que las recomendaciones aún pueden no ser totalmente personalizadas y precisas para cada usuario.

### **Añadiendo sugerencia a los títulos mal escritos**

In [312]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo_sug = df[['title', 'actors', 'keywords', 'genres', 'director']].copy()
# Se realiza un preprocesamiento en la columna "genres" para separar los géneros y convertirlos en palabras individuales
modelo_sug['genres'] = modelo_sug['genres'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
# Se realiza un preprocesamiento en la columna "director" para separar los géneros y convertirlos en palabras individuales
modelo_sug['director'] = modelo_sug['director'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
# Se realiza un preprocesamiento en la columna "actors" para separar los géneros y convertirlos en palabras individuales
modelo_sug['actors'] = modelo_sug['actors'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
# Se realiza un preprocesamiento en la columna "keywords" para separar los géneros y convertirlos en palabras individuales
modelo_sug['keywords'] = modelo_sug['keywords'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))

In [314]:
# Se crea una instancia de la clase TfidfVectorizer con los parámetros deseados
tfidf_sug = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
# Aplicar la transformación TF-IDF al texto contenido en las columnas "keywords", "genres", "actors" y "director" del dataframe 'modelo_sug'
tfidf_matriz_sug = tfidf_sug.fit_transform(modelo_sug['keywords'] + ' ' + modelo_sug['genres'] + ' ' + modelo_sug['director'] + ' ' + modelo_sug['actors'])

In [315]:
def suggest_movie_title(movie_title):
  # Obtener una lista de todos los títulos de películas disponibles
  available_movie_titles = modelo_sug['title'].tolist()
  # Encontrar las coincidencias más cercanas al título ingresado
  matches = process.extract(movie_title, available_movie_titles, limit=5)
  # Obtener las mejores coincidencias y sus puntajes de similitud
  best_matches = [match[0] for match in matches]
  best_match_scores = [match[1] for match in matches]
  # Verificar si hay una coincidencia lo suficientemente cercana
  if best_match_scores[0] >= 70:
    suggestion = best_matches[0]
    return f"Quizás quisiste decir: {suggestion}"
  else:
    return "No se encontraron coincidencias cercanas."

In [316]:
def get_recomendacion_msug(titulo):
  # Se crea un objeto 'indices' que mapea los títulos de las películas a sus índices correspondientes en el DataFrame 'modelo_sug'
  indices = pd.Series(modelo_sug.index, index=modelo_sug['title']).drop_duplicates()
  # Se busca el índice de la película de entrada utilizando el objeto 'indices'
  idx = pd.Series(indices[titulo]) if titulo in indices else None
  # Si la película de entrada no se encuentra en el DataFrame, se verifica si hay una sugerencia de título
  if idx is None:
    suggestion = suggest_movie_title(titulo)
    if suggestion != "No se encontraron coincidencias cercanas.":
      return suggestion
    else:
      return "Película no encontrada"
  # Si el título de la película está duplicado, se devuelve el índice de la primera aparición del título en el DataFrame
  if modelo_sug.duplicated(['title']).any():
    primer_idx = modelo_sug[modelo_sug['title'] == titulo].index[0]
    if not idx.equals(pd.Series(primer_idx)):
      idx = pd.Series(primer_idx)
  # Se calcula la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
  simil = sorted(enumerate(cosine_similarity(tfidf_matriz_sug[idx], tfidf_matriz_sug).flatten()), key=lambda x: x[1], reverse=True)[1:6]
  # Se obtienen los títulos de las películas más similares utilizando el índice de cada película
  recomendaciones = modelo_sug.iloc[[i[0] for i in simil], :]['title'].tolist()
  # Se devuelve la lista de títulos de las películas recomendadas
  return recomendaciones

In [318]:
# Prueba #1
get_recomendacion_msug('The Dark Knight Rises')

['The Dark Knight',
 'Batman Begins',
 'Interstellar',
 'The Wolf of Wall Street',
 'Inception']

In [319]:
# Prueba #1 Título mal escrito
get_recomendacion_msug('The Dark Knigth Rises')

'Quizás quisiste decir: The Dark Knight Rises'

In [321]:
# Prueba #2
get_recomendacion_msug('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Return of the Jedi',
 'Empire of Dreams: The Story of the Star Wars Trilogy',
 'Elstree 1976']

In [323]:
# Prueba #2 Título mal escrito
get_recomendacion_msug('Stars Wars')

'Quizás quisiste decir: Star Wars'

## **Modelo de Recomendación de Películas basado en Calificaciones Ponderadas**

Para este modelo se considerará el uso de la calificación ponderada de IMDB (también conocida como "Weighted Rating" o "wr") es un método para calcular la calificación de una película que tiene en cuenta tanto la calificación promedio de la película como el número de votos que ha recibido. La fórmula para calcular la calificación ponderada de IMDB es la siguiente:

![wr](https://1.bp.blogspot.com/-IwW-xX59Hi4/YF7TyvzmM7I/AAAAAAAAdkg/34Mpp3aW5LAsy561icqkdDEsq_O2ZgI9gCLcBGAsYHQ/s762/weight-avg.png)

donde:

- R es la calificación promedio de la película (con una escala de 0 a 10).
- v es el número de votos que ha recibido la película.
- m es el número mínimo de votos requerido para que una película sea considerada en el cálculo (generalmente se fija en 25).
- C es la calificación promedio de todas las películas en la base de datos de IMDB.

In [ ]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo5 = df[['genres', 'title', 'vote_count', 'vote_average', 'director']].copy()

In [ ]:
# Procesar los datos
modelo5['genres'] = modelo5['genres'].fillna('')
modelo5['genres'] = modelo5['genres'].str.split(',')
genres = pd.Series([genre for genres in modelo5['genres'] for genre in genres]).unique()
for genre in genres:
  modelo5[genre] = modelo5['genres'].apply(lambda x: 1 if genre in x else 0)
# Agregar una nueva columna con los nombres de los directores
modelo5['directores'] = modelo5['director'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

In [ ]:
# Calificación promedio de la película
v = modelo5['vote_count']
# Número de votos que ha recibido la película
R = modelo5['vote_average']
# Número mínimo de votos requerido
m = 25
# Calcular la calificación promedio de todas las películas
C = modelo5['vote_average'].mean()
# Calcular la calificación ponderada de las películas y agregarla como una nueva columna
modelo5['calificacion_ponderada'] = ((v / (v + m)) * R) + ((m / (v + m)) * C)

In [ ]:
# Imprimir las primeras 5 filas de "credits_csv"
modelo5.head()

,genres,title,vote_count,vote_average,director,animation,comedy,family,adventure,fantasy,...,mystery,war,foreign,music,documentary,western,tvmovie,,directores,calificacion_ponderada
0,"[animation, comedy, family]",Toy Story,5415.0,7.7,John Lasseter,1,1,1,0,0,...,0,0,0,0,0,0,0,0,[John Lasseter],7.690460
1,"[adventure, fantasy, family]",Jumanji,2413.0,6.9,Joe Johnston,0,0,1,1,1,...,0,0,0,0,0,0,0,0,[Joe Johnston],6.886918
2,"[romance, comedy]",Grumpier Old Men,92.0,6.5,Howard Deutch,0,1,0,0,0,...,0,0,0,0,0,0,0,0,[Howard Deutch],6.312862
3,"[comedy, drama, romance]",Waiting to Exhale,34.0,6.1,Forest Whitaker,0,1,0,0,0,...,0,0,0,0,0,0,0,0,[Forest Whitaker],5.898388
4,[comedy],Father of the Bride Part II,173.0,5.7,Charles Shyer,0,1,0,0,0,...,0,0,0,0,0,0,0,0,[Charles Shyer],5.690429


In [ ]:
# Función para recomendar películas
def get_recomendacion_m5(titulo):
  # Obtener la fila correspondiente al título de la película que se está recomendando
  pelicula = modelo5.loc[modelo5['title'] == titulo].iloc[0]
  # Calcular la similitud entre las películas
  modelo5['similitud'] = modelo5.apply(lambda x: (x[genres] * pelicula[genres]).sum() / (x[genres].sum() + pelicula[genres].sum()), axis=1)
  # Ordenar las películas por similitud y obtener las 5 mejores recomendaciones
  recomendaciones = modelo5.sort_values('similitud', ascending=False).head(6)[1:]
  # Calcular el peso adicional para las películas con una calificación ponderada alta y agregarlo a las recomendaciones
  recomendaciones['peso_calificacion'] = recomendaciones['calificacion_ponderada'].apply(lambda x: 1 if x >= pelicula['calificacion_ponderada'] else 0)
  # Calcular el peso adicional para las películas con el mismo director que la película que se está recomendando y agregarlo a las recomendaciones
  recomendaciones['peso_director'] = recomendaciones.apply(lambda x: len(set(x['directores']).intersection(set(pelicula['directores']))) / len(set(x['directores']).union(set(pelicula['directores']))), axis=1)
  # Calcular la puntuación total para cada recomendación y obtener las 5 mejores recomendaciones
  recomendaciones['peso_total'] = recomendaciones['similitud'] + (recomendaciones['peso_calificacion'] * 0.2) + (recomendaciones['peso_director'] * 0.2)
  recomendaciones = recomendaciones.sort_values('peso_total', ascending=False).head(5)

  return recomendaciones['title'].tolist()

In [ ]:
# Prueba #2
get_recomendacion_m5('The Dark Knight Rises')

['Death Wish', 'Pusher', 'After The Storm', 'Sin Nombre', 'The Asphalt Jungle']

In [ ]:
# Prueba #2
get_recomendacion_m5('The Avengers')

['Three Strangers',
 "Twilight's Last Gleaming",
 "Don't Say a Word",
 'Survivor',
 'Dangerous Company']

In [ ]:
# Prueba #3 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m5('Wonder Woman')

['Wonder Woman',
 'DC Showcase: Catwoman',
 'Justice League: Throne of Atlantis',
 'Macross Plus: Movie Edition',
 'Green Lantern: Emerald Knights']

In [ ]:
# Prueba #4 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m5('Titanic')

['Él',
 'The Bridesmaid',
 'Year of the Jellyfish',
 'Live Flesh',
 'Cruel Intentions 2']

Un inconveniente con este modelo es que las recomendaciones no son tan efectivas y el tiempo de respuesta supera el minuto y medio con un gasto aproximado de un 1 GB de RAM por consulta lo que supera el límite que ofrece Render para su servicio gratuito.

## **Comparación de resultados**

### Recomendación basado en contenido

#### Película ```The Dark Knight Rises```

In [148]:
# Película llamada "The Dark Knight Rises"
movie_title = "The Dark Knight Rises"

# Obtener las recomendaciones de los tres casos
modelo1 = get_recomendacion_m1(movie_title)
modelo2 = get_recomendacion_m2(movie_title)
modelo3 = get_recomendacion_m3(movie_title)
modelo4 = get_recomendacion_m4(movie_title)
modelo5 = get_recomendacion_m5(movie_title)

# Crear un DataFrame con las recomendaciones
recs_contenido = pd.DataFrame({
    'Modelo1 (title, overview)': modelo1,
    'Modelo2 (title, keywords)': modelo2,
    'Modelo3 (title, overview, genres)': modelo3,
    'Modelo4 (title, overview, genres, director)': modelo4,
    'Modelo5 (title, overview, genres, director, actors)': modelo5
})

recs_contenido

,"Modelo1 (title, overview)","Modelo2 (title, keywords)","Modelo3 (title, overview, genres)","Modelo4 (title, overview, genres, director)","Modelo5 (title, overview, genres, director, actors)"
0,The Dark Knight,The Dark Knight,The Dark Knight,The Dark Knight,The Dark Knight
1,Batman Forever,Batman Forever,Batman Forever,Batman Forever,Batman Begins
2,Batman,Batman,Batman Returns,Batman Returns,Interstellar
3,Batman Returns,Batman Returns,Batman,Batman,The Wolf of Wall Street
4,Batman: Under the Red Hood,Batman: Under the Red Hood,Batman: Under the Red Hood,Batman Begins,Inception


#### Película ```The Avengers```

In [170]:
# Película llamada "The Avengers"
movie_title = "The Avengers"

# Obtener las recomendaciones de los tres casos
modelo1 = get_recomendacion_m1(movie_title)
modelo2 = get_recomendacion_m2(movie_title)
modelo3 = get_recomendacion_m3(movie_title)
modelo4 = get_recomendacion_m4(movie_title)
modelo5 = get_recomendacion_m5(movie_title)

# Crear un DataFrame con las recomendaciones
recs_contenido = pd.DataFrame({
    'Modelo1 (title, overview)': modelo1,
    'Modelo2 (title, keywords)': modelo2,
    'Modelo3 (title, overview, genres)': modelo3,
    'Modelo4 (title, overview, genres, director)': modelo4,
    'Modelo5 (title, overview, genres, director, actors)': modelo5
})

recs_contenido

,"Modelo1 (title, overview)","Modelo2 (title, keywords)","Modelo3 (title, overview, genres)","Modelo4 (title, overview, genres, director)","Modelo5 (title, overview, genres, director, actors)"
0,The Work and the Glory,The Work and the Glory,The Work and the Glory,The Work and the Glory,The Work and the Glory
1,Sir Arne's Treasure,Monty Python and the Holy Grail,Sir Arne's Treasure,The Right Kind of Wrong,Hysterical Blindness
2,Monty Python and the Holy Grail,Sir Arne's Treasure,Monty Python and the Holy Grail,Benny & Joon,Eddie Izzard: Unrepeatable
3,A Beast at Bay,A Beast at Bay,A Beast at Bay,Diabolique,Eddie Izzard: Circle
4,Enigma,Enigma,Enigma,Sir Arne's Treasure,The Frightened City


#### Película ```Wonder Woman```

In [191]:
# Película llamada "Wonder Woman"
movie_title = "Wonder Woman"

# Obtener las recomendaciones de los tres casos
modelo1 = get_recomendacion_m1(movie_title)
modelo2 = get_recomendacion_m2(movie_title)
modelo3 = get_recomendacion_m3(movie_title)
modelo4 = get_recomendacion_m4(movie_title)
modelo5 = get_recomendacion_m5(movie_title)

# Crear un DataFrame con las recomendaciones
recs_contenido = pd.DataFrame({
    'Modelo1 (title, overview)': modelo1,
    'Modelo2 (title, keywords)': modelo2,
    'Modelo3 (title, overview, genres)': modelo3,
    'Modelo4 (title, overview, genres, director)': modelo4,
    'Modelo5 (title, overview, genres, director, actors)': modelo5
})

recs_contenido

,"Modelo1 (title, overview)","Modelo2 (title, keywords)","Modelo3 (title, overview, genres)","Modelo4 (title, overview, genres, director)","Modelo5 (title, overview, genres, director, actors)"
0,Wide Eyed and Legless,Wide Eyed and Legless,Wide Eyed and Legless,DC Showcase: Catwoman,DC Showcase: Catwoman
1,Trevor Noah: The Daywalker,Trevor Noah: The Daywalker,Trevor Noah: The Daywalker,Green Lantern: First Flight,Superman/Batman: Apocalypse
2,The Machinist,The Machinist,The Machinist,Wide Eyed and Legless,Green Lantern: Emerald Knights
3,Diana,Diana,Diana,Green Lantern: Emerald Knights,Green Lantern: First Flight
4,Our Dancing Daughters,Our Dancing Daughters,Our Dancing Daughters,Superman: Doomsday,Superman: Doomsday


#### Película ```Titanic```

In [207]:
# Película llamada "Titanic"
movie_title = "Titanic"

# Obtener las recomendaciones de los tres casos
modelo1 = get_recomendacion_m1(movie_title)
modelo2 = get_recomendacion_m2(movie_title)
modelo3 = get_recomendacion_m3(movie_title)
modelo4 = get_recomendacion_m4(movie_title)
modelo5 = get_recomendacion_m5(movie_title)

# Crear un DataFrame con las recomendaciones
recs_contenido = pd.DataFrame({
    'Modelo1 (title, overview)': modelo1,
    'Modelo2 (title, keywords)': modelo2,
    'Modelo3 (title, overview, genres)': modelo3,
    'Modelo4 (title, overview, genres, director)': modelo4,
    'Modelo5 (title, overview, genres, director, actors)': modelo5
})

recs_contenido

,"Modelo1 (title, overview)","Modelo2 (title, keywords)","Modelo3 (title, overview, genres)","Modelo4 (title, overview, genres, director)","Modelo5 (title, overview, genres, director, actors)"
0,Grantham and Rose,Grantham and Rose,Grantham and Rose,Ghosts of the Abyss,Aliens of the Deep
1,Take Care of the Women!,Take Care of the Women!,Titanic,Grantham and Rose,Titanic: The Final Word with James Cameron
2,The Legend of 1900,The Legend of 1900,Take Care of the Women!,Titanic,Ghosts of the Abyss
3,Raise the Titanic,Raise the Titanic,Raise the Titanic,The Legend of 1900,The Bounty
4,Beyond the Poseidon Adventure,Beyond the Poseidon Adventure,The Legend of 1900,Deadly Voyage,The Departed


#### Película ```Stars Wars```

In [269]:
# Película llamada "Stars Wars"
movie_title = "Stars Wars"

# Obtener las recomendaciones de los tres casos
modelo1 = get_recomendacion_m1(movie_title)
modelo2 = get_recomendacion_m2(movie_title)
modelo3 = get_recomendacion_m3(movie_title)
modelo4 = get_recomendacion_m4(movie_title)
modelo5 = get_recomendacion_m5(movie_title)

# Crear un DataFrame con las recomendaciones
recs_contenido = pd.DataFrame({
    'Modelo1 (title, overview)': modelo1,
    'Modelo2 (title, keywords)': modelo2,
    'Modelo3 (title, overview, genres)': modelo3,
    'Modelo4 (title, overview, genres, director)': modelo4,
    'Modelo5 (title, overview, genres, director, actors)': modelo5
})

recs_contenido

,"Modelo1 (title, overview)","Modelo2 (title, keywords)","Modelo3 (title, overview, genres)","Modelo4 (title, overview, genres, director)","Modelo5 (title, overview, genres, director, actors)"
0,The Empire Strikes Back,The Empire Strikes Back,The Empire Strikes Back,The Empire Strikes Back,The Empire Strikes Back
1,The Star Wars Holiday Special,The Star Wars Holiday Special,The Star Wars Holiday Special,The Star Wars Holiday Special,The Star Wars Holiday Special
2,Star Wars: The Force Awakens,Star Wars: The Force Awakens,Return of the Jedi,Return of the Jedi,Return of the Jedi
3,Return of the Jedi,Return of the Jedi,Star Wars: The Force Awakens,Star Wars: The Force Awakens,Empire of Dreams: The Story of the Star Wars T...
4,Samson and the Seven Miracles of the World,Samson and the Seven Miracles of the World,Samson and the Seven Miracles of the World,Star Wars: Episode I - The Phantom Menace,Elstree 1976


#### Película ```The Hunder Games```

In [296]:
# Película llamada "The Hunder Games"
movie_title = "The Hunder Games"

# Obtener las recomendaciones de los tres casos
modelo1 = get_recomendacion_m1(movie_title)
modelo2 = get_recomendacion_m2(movie_title)
modelo3 = get_recomendacion_m3(movie_title)
modelo4 = get_recomendacion_m4(movie_title)
modelo5 = get_recomendacion_m5(movie_title)

# Crear un DataFrame con las recomendaciones
recs_contenido = pd.DataFrame({
    'Modelo1 (title, overview)': modelo1,
    'Modelo2 (title, keywords)': modelo2,
    'Modelo3 (title, overview, genres)': modelo3,
    'Modelo4 (title, overview, genres, director)': modelo4,
    'Modelo5 (title, overview, genres, director, actors)': modelo5
})

recs_contenido

,"Modelo1 (title, overview)","Modelo2 (title, keywords)","Modelo3 (title, overview, genres)","Modelo4 (title, overview, genres, director)","Modelo5 (title, overview, genres, director, actors)"
0,The Hunger Games: Catching Fire,The Hunger Games: Catching Fire,The Hunger Games: Catching Fire,The Hunger Games: Catching Fire,The Hunger Games: Catching Fire
1,The Hunger Games: Mockingjay - Part 2,The Hunger Games: Mockingjay - Part 2,The Hunger Games: Mockingjay - Part 2,The Hunger Games: Mockingjay - Part 2,The Hunger Games: Mockingjay - Part 2
2,The Starving Games,The Starving Games,The Starving Games,The Starving Games,The Hunger Games: Mockingjay - Part 1
3,The Hungover Games,The Hungover Games,The Hungover Games,The Hungover Games,Buster's Mal Heart
4,Indie Game: The Movie,Indie Game: The Movie,The Hunger Games: Mockingjay - Part 1,The Hunger Games: Mockingjay - Part 1,Quarantine 2: Terminal


## **Recomendaciones finales**

Aquí te presento algunas recomendaciones para mejorar los modelos presentados en este notebook, que por limitaciones en cuanto al consumo de la RAM no pudieron ser empleados considerando el total de la data.

- **Optimizar los parámetros de TfidfVectorizer**: El modelo utiliza la clase TfidfVectorizer para convertir los textos de los resúmenes en una matriz de características TF-IDF. Se podría experimentar con diferentes valores de los parámetros, como el conjunto de stop words, el rango de n-gramas y los valores de min_df y max_df para encontrar los valores óptimos para el modelo.

- **Considerar el uso de técnicas de reducción de dimensionalidad**: La matriz de características generada por TfidfVectorizer podría tener una alta dimensionalidad, lo que podría afectar negativamente el rendimiento del modelo y aumentar el tiempo de procesamiento. Se podría considerar el uso de técnicas de reducción de dimensionalidad, como PCA o t-SNE, para reducir la dimensionalidad de la matriz de características antes de aplicar el modelo.


## **Modelo empleado en el proyecto**

Pese a que el modelo que presentó una mejor recomendación fue el ```Modelo5``` junto con la sugerencia a los títulos mal escritos ingresados por el usuario. La elección del modelo final para este proyecto se basó en su capacidad para adaptarse a las limitaciones de consumo de recursos. Por lo tanto, el modelo seleccionado fue el ```Modelo1``` debido a su menor costo computacional, a pesar de que su precisión es menor que la de otros modelos presentados en este notebook.

In [297]:
# Imprimir las primeras 5 filas del Dataframe "merged"
df[['title', 'overview_clean']].head()

,title,overview_clean
0,Toy Story,led by woody andys toy live happily in his roo...
1,Jumanji,when sibling judy and peter discover an enchan...
2,Grumpier Old Men,a family wedding reignites the ancient feud be...
3,Waiting to Exhale,cheated on mistreated and stepped on the woman...
4,Father of the Bride Part II,just when george bank ha recovered from his da...


In [298]:
# Se exporta las columnas necesarias para el Sistema de Recomendación
modelo = df[['title', 'overview_clean']]
modelo.to_parquet('movies_recommendations.parquet')